<a href="https://colab.research.google.com/github/jayashalakshani/Real-Time-Social-Media-Sentiment-and-Trend-Analysis-Platform/blob/main/Speed_Layer_(Real_Time_Processing_with_Spark_%2B_Redis).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark Setup in Colab

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!pip install -q findspark pymongo[srv] redis pyspark transformers torch redis
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
ls -l spark-3.5.0-bin-hadoop3.tgz

-rw-r--r-- 1 root root 400395283 Sep  9  2023 spark-3.5.0-bin-hadoop3.tgz


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

# Initialize Spark

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MongoDBToRedisSpeedLayer") \
    .getOrCreate()


# Connect to MongoDB and Redis

In [ ]:
from pyspark.sql import SparkSession
import redis
from pymongo import MongoClient
from transformers import pipeline
import json
import time
from google.colab import userdata

In [ ]:
# Replace these with your credentials
username = userdata.get('mongodb_username')
password = userdata.get('mongodb_pw')
cluster_url = "cluster0.8ad48r1.mongodb.net"

In [ ]:
# Full URI
uri = f"mongodb+srv://{username}:{password}@{cluster_url}/?retryWrites=true&w=majority&appName=Cluster0"

In [19]:
# MongoDB connection
client = MongoClient(uri)
db = client['social_media_analytics']
collection_1 = db['mastodon_sentiment_data']
collection_2 = db['youtube_sentiment_collection']

In [ ]:
# Redis connection
redis_client = redis.Redis(
    host='redis-15930.c212.ap-south-1-1.ec2.redns.redis-cloud.com',
    port=15930,
    username='default',
    password= userdata.get('redis_password'),
    decode_responses=True
)

In [ ]:
# Checking redis
redis_client.set('foo', 'bar')

True

In [ ]:
# Sentiment model
sentiment_pipeline = pipeline('sentiment-analysis', model="distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


# Define Micro-Batch Streaming

In [22]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from pyspark.sql import Row
import json
import time



📜 Mastodon: Processing 50 comments
+----------------------+--------------------------------------------------+-----------------------+--------------------------------------------------+
|                   tag|                                              text|             created_at|                                          post_url|
+----------------------+--------------------------------------------------+-----------------------+--------------------------------------------------+
|   #screenshotsaturday| thank thats really encouraging hear exactly going|    2025-04-27 07:01:59|https://mastodon.gamedev.place/@HaafAndHammar/1...|
|   #screenshotsaturday|icons right recipe buttons indicate ingredient ...|    2025-04-27 03:40:51|  https://kind.social/@empyreon/114407830618400411|
|   #screenshotsaturday|really enjoy boardgamebecamereal look youre making|    2025-04-27 01:50:55|https://mastodon.gamedev.place/@HaafAndHammar/1...|
|        #fantasticfour|                                  

In [ ]:
# Create schemas
mastodon_schema = StructType([
    StructField("tag", StringType(), True),
    StructField("text", StringType(), True),
    StructField("created_at", TimestampType(), True),
    StructField("post_url", StringType(), True)
])

youtube_schema = StructType([
    StructField("tag", StringType(), True),
    StructField("text", StringType(), True),
    StructField("published_at", StringType(), True),
    StructField("video_id", StringType(), True),
    StructField("video_title", StringType(), True)
])

In [ ]:
# Track processed IDs separately
processed_mastodon_ids = set()
processed_youtube_ids = set()

while True:
    # Step 1: Process Mastodon comments
    mastodon_comments = collection_1.find().sort('created_at', -1).limit(50)

    mastodon_list = []
    for comment in mastodon_comments:
        comment_id = str(comment['_id'])
        if comment_id not in processed_mastodon_ids:
            mastodon_list.append(Row(
                tag=comment.get('tag', ''),
                text=comment.get('text', ''),
                created_at=comment.get('created_at'),
                post_url=comment.get('post_url', '')
            ))
            processed_mastodon_ids.add(comment_id)

    if mastodon_list:
        mastodon_df = spark.createDataFrame(mastodon_list, schema=mastodon_schema)
        print(f"\n📜 Mastodon: Processing {len(mastodon_list)} comments")
        mastodon_df.show(truncate=50)

        for row in mastodon_df.collect():
            text = row['text']
            if text:
                sentiment = sentiment_pipeline(text[:512])[0]
                label = sentiment['label']
                confidence = sentiment['score']

                cache_key = f"mastodon:{row['post_url']}"
                cache_value = {
                    'platform': 'mastodon',
                    'tag': row['tag'],
                    'text': text,
                    'sentiment': label,
                    'confidence': confidence,
                    'created_at': str(row['created_at']),
                    'post_url': row['post_url']
                }
                redis_client.set(cache_key, json.dumps(cache_value), ex=600)
    else:
        print("✅ Mastodon: No new comments found.")

    # Step 2: Process YouTube comments
    youtube_comments = collection_2.find().sort('published_at', -1).limit(50)

    youtube_list = []
    for comment in youtube_comments:
        comment_id = str(comment['_id'])
        if comment_id not in processed_youtube_ids:
            youtube_list.append(Row(
                tag=comment.get('tag', ''),
                text=comment.get('text', ''),
                published_at=comment.get('published_at'),
                video_id=comment.get('video_id', ''),
                video_title=comment.get('video_title', '')
            ))
            processed_youtube_ids.add(comment_id)

    if youtube_list:
        youtube_df = spark.createDataFrame(youtube_list, schema=youtube_schema)
        print(f"\n📜 YouTube: Processing {len(youtube_list)} comments")
        youtube_df.show(truncate=50)

        for row in youtube_df.collect():
            text = row['text']
            if text:
                sentiment = sentiment_pipeline(text[:512])[0]
                label = sentiment['label']
                confidence = sentiment['score']

                cache_key = f"youtube:{row['video_id']}"
                cache_value = {
                    'platform': 'youtube',
                    'tag': row['tag'],
                    'text': text,
                    'sentiment': label,
                    'confidence': confidence,
                    'published_at': str(row['published_at']),
                    'video_id': row['video_id'],
                    'video_title': row['video_title']
                }
                redis_client.set(cache_key, json.dumps(cache_value), ex=600)
    else:
        print("✅ YouTube: No new comments found.")

    # 💥 Stop condition: if both Mastodon and YouTube have no new comments
    if not mastodon_list and not youtube_list:
        print("\n🎉 No more new data in both collections. Exiting...")
        break

    time.sleep(10)  # Sleep for 10 seconds before the next iteration

In [23]:
# List all keys
keys = redis_client.keys('*')
print("All keys in Redis:", keys)

# Pick one key and get its value
for key in keys:
    # key is already a string, no need to decode
    value = redis_client.get(key)
    if isinstance(value, bytes):
        value = value.decode('utf-8')  # Decode if value is in bytes

    print(f"\nKey: {key}")
    print(f"Value: {value}")


All keys in Redis: ['youtube:7Qudtm702v4', 'mastodon:https://chaos.social/@minkorrekt/114354195908604624', 'youtube:bthtAJiE3iE', 'mastodon:https://kolektiva.social/@johnbrowntypebeats/114354013957301701', 'mastodon:https://mastodon.social/@OldSchool63/114356586051075352', 'mastodon:https://mastodon.gamedev.place/@HaafAndHammar/114406812980104136', 'mastodon:https://kind.social/@empyreon/114407830618400411', 'youtube:DJf04cdgk70', 'mastodon:https://mastodon.social/@limebar/114354114571482860', 'mastodon:https://gardenstate.social/@BlowUpRadio/114353946008727230', 'youtube:BD4HQmczHoo', 'youtube:MkMT5HCjwnU', 'mastodon:https://mastodon.social/@SKleefeld/114400415439556358', 'youtube:tNsuxrUbOpk', 'mastodon:https://c.im/@jake4480/114394247589967767', 'mastodon:https://nham.co.uk/2025/04/11714/', 'mastodon:https://cupoftea.social/@Gaolaitch/114383865907889514', 'mastodon:https://mastodon.social/@geoff_eg/114395574072985295', 'mastodon:https://esq.social/@D_J_Nathanson/114353709962713077',

In [20]:
# # remove data in redis
# redis_client.flushdb()

True